In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors
import matplotlib
# matplotlib.artist.getp(fig.patch) # https://matplotlib.org/stable/tutorials/intermediate/artists.html#sphx-glr-tutorials-intermediate-artists-py
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
# usually this notebook is loaded as top-level-module and relative import of chsimpy does not work.
# so we provide the path to the chsimpy package manually
import pathlib
import sys

try:
    import chsimpy
except ImportError:
    _parentdir = pathlib.Path("./").resolve().parent
    sys.path.insert(0, str(_parentdir))
    import chsimpy
    #sys.path.remove(str(_parentdir))

# auto reload if chsimpy code changed
%load_ext autoreload
%autoreload 2

In [ ]:
dirfiles = '../_output/paper-1k'
rawfile = f"{dirfiles}/experiment-17022023-114325-raw.csv"
sysinfofile = f"{dirfiles}/experiment-17022023-114325-sysinfo.csv"
aggfile = f"{dirfiles}/experiment-17022023-114325-agg.csv"

In [ ]:
dfagg = pd.read_csv(aggfile, index_col=0)
dfraw = pd.read_csv(rawfile, index_col=['id'])
dfraw = dfraw.drop(columns=dfraw.columns[0]) # drops unnamed column
sysinfo = pd.read_csv(sysinfofile, header=None).T

In [ ]:
dfsys = sysinfo.iloc[0].str.split('=')
keys = [ x[0] for x in dfsys]
values = [ x[1].replace("'", '') for x in dfsys]
dfsys = pd.DataFrame(values, index=keys)

In [ ]:
dfagg

In [ ]:
dfraw

In [ ]:
cacb = np.abs(dfraw.ca-dfraw.cb)
#
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12, 10))
#
# nrow, ncol, index
axs[0,0].plot(dfraw.A0, cacb, '.', ms=2)
axs[0,0].set_xlabel('A0')
axs[0,0].set_ylabel('|c_a - c_b|')

axs[0,1].plot(dfraw.A1, cacb, '.', ms=2)
axs[0,1].set_xlabel('A1')
axs[0,1].set_ylabel('|c_a - c_b|')

axs[1,0].plot(dfraw.A0, dfraw.tsep, '.', ms=2, c='purple')
axs[1,0].set_xlabel('A0')
axs[1,0].set_ylabel('t_sep')

axs[1,1].plot(dfraw.A1, dfraw.tsep, '.', ms=2, c='purple')
axs[1,1].set_xlabel('A1')
axs[1,1].set_ylabel('t_sep')

axs[2,0].plot(cacb, dfraw.tsep, '.', ms=2, c='y')
axs[2,0].set_xlabel('|c_a - c_b|')
axs[2,0].set_ylabel('t_sep')

dfcacb = pd.melt(dfraw[['cb','ca']])
sns.boxplot(y="variable", x="value", data=dfcacb, ax=axs[2,1])
#axs[2,1].plot(cacb, dfraw.tsep-dfraw.tau0, '.', ms=2, c='y')
axs[2,1].set_xlabel('c_* for T = 923.15K')
axs[2,1].set_ylabel('')

display(fig)
matplotlib.pyplot.close()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(8, 5), sharex=True)
#
# nrow, ncol, index
sns.kdeplot(dfraw.tsep, ax=axs[0])
axs[0].set_xlabel('')
axs[0].set_ylabel('dist')

sns.boxplot(dfraw.tsep, orient='h', ax=axs[1])
axs[1].set_xlabel('iterations')
axs[1].set_ylabel('')

display(fig)
matplotlib.pyplot.close()

In [ ]:
import glob, os
# solution-17022023-114325-run106.E.csv.bz2
files_E = glob.glob(dirfiles+r'/solution-17022023-114325-run*.E.csv.bz2')

# listE = []

# for filename in files_E:
#     #E = pd.read_csv(f"{dirfiles}/{filename}", sep=',', header=None, compression='bz2').values
#     E = chsimpy.utils.csv_load_matrix(filename)
#     listE.append(E)

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  
# Axes3D import has side effects, it enables using projection='3d' in add_subplot
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x = dfraw.A0
y = dfraw.A1
X, Y = np.meshgrid(x, y)
zs = np.array(np.ravel(dfraw.tsep))
Z = zs.reshape(X.shape)

ax.plot_surface(X, Y, Z)